In [1]:
import numpy as np
import pandas as pd
from electroviz.core.experiment import Experiment
from electroviz.utils.cross_corr import cross_corr

exp = Experiment("E:/RG001/")

stimuli = exp.stimuli
pop0 = exp.populations[0]
pop0 = pop0.remove(pop0.units["spike_rate"] < 1)
pop0_sort = pop0.sort("depth")

# xcorr_df = pd.DataFrame()

# for cortex_idx, cortex_unit in enumerate(pop0_sort[:90]):
#     for lgn_idx, lgn_unit in enumerate(pop0_sort[185:270]):
#         xcorr = cross_corr(cortex_unit, lgn_unit, stimuli[4], rand_iters=10)
#         baseline = xcorr[90:100].mean()
#         response = xcorr[102:106].mean()
#         xc = np.around(response / baseline, decimals=2)
#         (peak,) = np.where(xcorr == np.max(xcorr))
#         df = pd.DataFrame({"cortex_idx":[cortex_idx], 
#                            "lgn_idx":[lgn_idx + 185], 
#                            "xcorr":[xc], 
#                            "peak":[peak[0] - 100]})
#         xcorr_df = pd.concat((xcorr_df, df))

In [6]:
from electroviz.core.experiment import Experiment
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.viz.summary import PairSummary

exp = Experiment("E:/RG001/")

stimuli = exp.stimuli
pop0 = exp.populations[0]
pop0 = pop0.remove(pop0.units["spike_rate"] < 1)
pop0_sort = pop0.sort("depth")

stims = [stimuli[2], stimuli[1], stimuli[4], stimuli[0]]

ctx_unit = pop0_sort[3]
ctx_kernels = [SparseNoiseKernel(cortex, stims[0]), 
               SparseNoiseKernel(cortex, stims[1]), 
               StaticGratingsKernel(cortex, stims[2]), 
               StaticGratingsKernel(cortex, stims[3])]

lgn_unit = pop0_sort[246]
lgn_kernels = [SparseNoiseKernel(lgn, stims[0]), 
               SparseNoiseKernel(lgn, stims[1]), 
               StaticGratingsKernel(lgn, stims[2]), 
               StaticGratingsKernel(lgn, stims[3])]

PairSummary(stims, ctx_unit, ctx_kernels, lgn_unit, lgn_kernels)